<a href="https://colab.research.google.com/github/RO-AD/waymo-od-motion-pred/blob/main/tutorial/4_dataset-structure/gp-dataset-structure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터셋 구조 분석 및 시각화

- 저번에는 공식 튜토리얼을 보고 진행했었다. `tf_example/` 하위의 데이터셋을 가지고 시각화를 진행했었는데, 아무래도 tf_example 파싱 부분이 마음에 안들어서 `scenario/` 데이터셋을 가지고 구조 분석 및 시각화를 다시 진행해보려고 한다.

- 총 3개의 proto 파일을 분석할 예정이다. **sceanario.proto** 에서 나머지 두 proto를 import 해서 사용하기 때문.
  - [scenario.proto](https://github.com/waymo-research/waymo-open-dataset/blob/master/src/waymo_open_dataset/protos/scenario.proto)
    ```python
    import "waymo_open_dataset/protos/compressed_lidar.proto";
    import "waymo_open_dataset/protos/map.proto";
    ```
  - [map.proto](https://github.com/waymo-research/waymo-open-dataset/blob/master/src/waymo_open_dataset/protos/map.proto)
  - [compressed_lidar.proto](https://github.com/waymo-research/waymo-open-dataset/blob/master/src/waymo_open_dataset/protos/compressed_lidar.proto)

- proto 파일 분석 전 알아야 할 것
  - proto 파일은 Protocol Buffers라는 구글에서 만든 직렬화 데이터 저장 방식.
  - message로 선언.
  - message 하위 요소들은 `{속성} {자료형} {이름}={넘버링};` 형태로 선언  
  - 속성의 종류
    - optional : 없어도 됨.
    - repeated : 리스트 형태로 구현됨
    - oneof : 하위 요소 중 하나 선택


## 환경세팅

In [4]:
from google.colab import drive
drive.mount('/content/drive')

# 구글 드라이브 위치. 마운트 잘 됐는지 확인.
GDRIVE_WOD_PATH = "/content/drive/MyDrive/waymo-od-dataset/motion_v_1_2_0/uncompressed"
VALIDSET_PATH   = GDRIVE_WOD_PATH + "/scenario/validation"

# 마운트 확인
!ls -al {VALIDSET_PATH} | wc -l

Mounted at /content/drive
151


In [5]:
%%capture

!pip3 install --upgrade pip
!pip install waymo-open-dataset-tf-2-11-0==1.5.1

## [1] 데이터셋 로드

In [6]:
!ls -al {VALIDSET_PATH} | head

total 40266256
-rw------- 1 root root 273686917 Apr 14 03:30 validation.tfrecord-00000-of-00150
-rw------- 1 root root 272253722 Apr 14 03:30 validation.tfrecord-00001-of-00150
-rw------- 1 root root 281894961 Apr 14 03:30 validation.tfrecord-00002-of-00150
-rw------- 1 root root 283858404 Apr 14 03:30 validation.tfrecord-00003-of-00150
-rw------- 1 root root 293786123 Apr 14 03:30 validation.tfrecord-00004-of-00150
-rw------- 1 root root 275962185 Apr 14 03:30 validation.tfrecord-00005-of-00150
-rw------- 1 root root 262424772 Apr 14 03:29 validation.tfrecord-00006-of-00150
-rw------- 1 root root 263608876 Apr 14 03:29 validation.tfrecord-00007-of-00150
-rw------- 1 root root 287419235 Apr 14 03:30 validation.tfrecord-00008-of-00150


In [10]:
import tensorflow as tf
from waymo_open_dataset.protos import scenario_pb2

# TFRecordDataset 클래스를 사용하여 데이터를 읽어들임
dataset = tf.data.TFRecordDataset(VALIDSET_PATH + '/validation.tfrecord-00000-of-00150')

for data in dataset:
  scenario = scenario_pb2.Scenario()
  scenario.ParseFromString(bytearray(data.numpy()))
  break

scenario.scenario_id

'19a486cd29abd7a7'

## [2] Scenario 1-depth 분석

- 앞서 scenario 인스턴스를 생성해서 데이터를 파싱했다. proto 파일에 작성된 구조는 다음과 같다.
  ```proto
  message Scenario {
    reserved 9;

    optional string scenario_id = 5;
    repeated double timestamps_seconds = 1;
    optional int32 current_time_index = 10;
    repeated Track tracks = 2;
    repeated DynamicMapState dynamic_map_states = 7;
    repeated MapFeature map_features = 8;
    optional int32 sdc_track_index = 6;
    repeated int32 objects_of_interest = 4;
    repeated RequiredPrediction tracks_to_predict = 11;
    repeated CompressedFrameLaserData compressed_frame_laser_data = 12;
  }
  ```
- string, double, int 등 단순한 자료형으로 선언되어있는 요소들을 먼저 본다.

In [24]:
# optional string scenario_id = 5;

# 시나리오 고유 넘버
scenario.scenario_id

'19a486cd29abd7a7'

In [25]:
# repeated double timestamps_seconds

# 본 시나리오에서 각 장면을 측정한 시각
# 총 9초. 과거(1초) + 미래(9초)
# 실제 testset에서는 앞 1초만 제공되어 뒤 8초를 예측해야 함.
scenario.timestamps_seconds

[0.0, 0.10004, 0.20005, 0.3, 0.40001, 0.49998, 0.59998, 0.69998, 0.79999, 0.89999, 0.99999, 1.09995, 1.19996, 1.29992, 1.39991, 1.49988, 1.59987, 1.69992, 1.79991, 1.89994, 1.99994, 2.09994, 2.19994, 2.29995, 2.39998, 2.49998, 2.59998, 2.69998, 2.79998, 2.89998, 2.99998, 3.10001, 3.20005, 3.30009, 3.40013, 3.50013, 3.60013, 3.70017, 3.80017, 3.90017, 4.00017, 4.10018, 4.20014, 4.30011, 4.40006, 4.50002, 4.59999, 4.69995, 4.79992, 4.89988, 4.99988, 5.09984, 5.19984, 5.29984, 5.39984, 5.49988, 5.59989, 5.69993, 5.79993, 5.89996, 6.0, 6.1, 6.2, 6.30003, 6.40003, 6.5, 6.6, 6.7, 6.8, 6.9, 6.99997, 7.09997, 7.19996, 7.29993, 7.39988, 7.49988, 7.59988, 7.69988, 7.79988, 7.89992, 7.99992, 8.09992, 8.19993, 8.29996, 8.39997, 8.49997, 8.59997, 8.69997, 8.79998, 8.89998, 8.99994]

In [27]:
# optional int32 current_time_index

# 전체 관측 중 '현재'라고 판단하는 시점의 index.
# 과거 1초가 거의 끝난 0.99999초 부분이 현재임을 알 수 있음.
print(scenario.current_time_index)
print(scenario.timestamps_seconds[scenario.current_time_index])

10
0.99999


In [36]:
# repeated Track tracks = 2;

# Map 내에 존재하는 Agent들의 상태정보를 가지고 있음.
# Track 구조에 대해서는 2depth에서 다시 언급
print(type(scenario.tracks))
print(type(scenario.tracks[0]))

<class 'google.protobuf.pyext._message.RepeatedCompositeContainer'>
<class 'waymo_open_dataset.protos.scenario_pb2.Track'>


In [37]:
# repeated DynamicMapState dynamic_map_states = 7;

# 맵 정보 중 동적인 요소에 대한 정보를 담고 있음.
# DynamicMapState 구조에 대해서는 2depth에서 다시 언급
print(type(scenario.dynamic_map_states))
print(type(scenario.dynamic_map_states[0]))

<class 'google.protobuf.pyext._message.RepeatedCompositeContainer'>
<class 'waymo_open_dataset.protos.scenario_pb2.DynamicMapState'>


In [39]:
# repeated MapFeature map_features = 8;

# 맵 정보에 대한 정보를 담고 있음(정적)
# MapFeature 구조에 대해서는 map.proto에서 다시 언급
print(type(scenario.map_features))
print(type(scenario.map_features[0]))

<class 'google.protobuf.pyext._message.RepeatedCompositeContainer'>
<class 'waymo_open_dataset.protos.map_pb2.MapFeature'>


In [40]:
# optional int32 sdc_track_index

# sdc : Self-Driving Car
# 각 장면에는 장면을 측정하고 있는 agent 존재
# Tracks 내 여러 agent 중 sdc_track_index에 해당하는 agent가 측정용 객체.
scenario.sdc_track_index

10

In [46]:
# repeated int32 objects_of_interest

# Track 내부에 obejct id 정보가 있는데, 관심있는 대상의 object id를 모아놓은 것.
# ('관심있다'에 대한 필요성 및 중요도는 아직 잘 모르겠음)
scenario.objects_of_interest

[]

In [47]:
# repeated RequiredPrediction tracks_to_predict

# 예측해야 할 track에 대한 정보를 담고있음.
# 이 외의 track은 예측 시 무시해도 됨.
scenario.tracks_to_predict

[track_index: 2
difficulty: LEVEL_1
]

In [48]:
# repeated CompressedFrameLaserData compressed_frame_laser_data

# 라이다 데이터가 담기는 공간이라고 함.
# 근데 제공된 데이터셋에는 라이다 데이터셋이 따로 제공됨.
# (안쓰는건가,,?)
scenario.compressed_frame_laser_data

[]

## [3] Scenario 2-depth 분석

- scenario 메세지 내부에 기본 자료형으로 선언되지 않은 요소들이 있었음.
  - repeated **Track tracks** = 2;
  - repeated **DynamicMapState dynamic_map_states** = 7;
  - repeated **MapFeature map_features** = 8;
    - MapFeature는 map.proto에 선언되어 있으므로 추후 다시 언급
  - repeated **RequiredPrediction tracks_to_predict**
  - repeated **CompressedFrameLaserData compressed_frame_laser_data**
    - CompressedFrameLaserData는 compressed_lidar.proto에 선언되어 있으므로 추후 다시 언급
- 이 요소들을 한단계 더 나아가 분석해보고자 함.

### [3-1] repeated Track tracks = 2;

```proto
message Track {
  enum ObjectType {
    TYPE_UNSET = 0;  // This is an invalid state that indicates an error.
    TYPE_VEHICLE = 1;
    TYPE_PEDESTRIAN = 2;
    TYPE_CYCLIST = 3;
    TYPE_OTHER = 4;
  }

  optional int32 id = 1;
  optional ObjectType object_type = 2;
  repeated ObjectState states = 3;
}
```

In [63]:
# repeated Track tracks = 2;

# Map 내에 존재하는 Agent들의 상태정보를 가지고 있음.
print(len(scenario.tracks))

track = scenario.tracks[0]
print(f'* Track id   : {track.id}')
print(f'* Track type : {track.object_type}') # TYPE_VEHICLE
print(f'      -> {track.object_type == track.TYPE_VEHICLE}')
print(type(track.states))
print(type(track.states[0])) # ObjectState에 대해서는 3depth에서 다룰 예정

11
* Track id   : 368
* Track type : 1
      -> True
<class 'google.protobuf.pyext._message.RepeatedCompositeContainer'>
<class 'waymo_open_dataset.protos.scenario_pb2.ObjectState'>


### [3-2] repeated DynamicMapState dynamic_map_states = 7;

```proto
message DynamicMapState {
  // The traffic signal states for all observed signals at this time step.
  repeated TrafficSignalLaneState lane_states = 1;
}
```

In [78]:
# repeated DynamicMapState dynamic_map_states = 7;

# 맵 정보 중 동적인 요소에 대한 정보를 담고 있음.
print(f'* dynamic map stats length: {len(scenario.dynamic_map_states)}')

dynamic_lane_states = scenario.dynamic_map_states[0].lane_states
print(f'* dynamic lane state length : {len(dynamic_lane_states)}')
dynamic_lane_states[0] # TrafficSignalLaneState에 대해서는 map.proto 에서 다시 언급

* dynamic map stats length: 91
* dynamic lane state length : 4


lane: 120
state: LANE_STATE_GO
stop_point {
  x: 8391.668352965295
  y: 7316.195065592189
  z: -14.848865196389456
}

### [3-3] repeated RequiredPrediction tracks_to_predict

```
message RequiredPrediction {
  enum DifficultyLevel {
    NONE = 0;
    LEVEL_1 = 1;
    LEVEL_2 = 2;
  }
  optional int32 track_index = 1;
  optional DifficultyLevel difficulty = 2;
}
```

In [73]:
# repeated RequiredPrediction tracks_to_predict

# 예측해야 할 track에 대한 정보를 담고있음.
# 이 외의 track은 예측 시 무시해도 됨.
print(f'* TTP length  : {len(scenario.tracks_to_predict)}')

ttp = scenario.tracks_to_predict[0]
print(f'    * Track Index : {ttp.track_index}')
print(f'    * Difficulty  : {ttp.difficulty}')

print(ttp)

* TTP length  : 1
    * Track Index : 2
    * Difficulty  : 1
track_index: 2
difficulty: LEVEL_1



## [4] Scenario 3-depth 분석

## [5] Map 분석

# 기타

In [49]:
# tutorial/4_dataset-structure/gp-dataset-structure.ipynb